In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 迁移：超参数调整

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-hyperparameter-tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-hyperparameter-tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-hyperparameter-tuning.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来调整自定义表格分类TensorFlow模型中的超参数。

了解更多关于[Migrate to Vertex AI](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)和[Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Hyperparameter`来创建和调整自定义训练模型。

您将学习如何使用Vertex AI SDK for Python在Docker容器中从Python脚本创建和调整自定义训练模型。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Training`
- `Vertex AI Hyperparameter Tuning`

执行的步骤包括：

- 为训练TensorFlow模型创建一个`Vertex AI`超参数调整作业。

数据集

本教程使用的数据集是波士顿房价数据集。本教程中使用的数据集版本已经集成在TensorFlow中。训练模型预测房屋的中位价格，单位为1K美元。

### 费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储的定价](https://cloud.google.com/storage/pricing)，以及使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用量生成费用估算。

## 安装

安装执行此笔记本所需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作:
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面: [查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。 了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时：运行下面的单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化 Python 的 Vertex SDK，用于您的项目和相应的存储桶。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

（aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4）

否则，指定`(None, None)`以使用一个容器映像在CPU上运行。

了解更多信息，请访问[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) 有关您所在地区的硬件加速器支持。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

设置用于训练和预测的预构建Docker容器镜像。


有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2-5"

if TRAIN_GPU:
    TRAIN_VERSION = "tf-gpu.{}".format(TF)
else:
    TRAIN_VERSION = "tf-cpu.{}".format(TF)
if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
MACHINE_TYPE = "n1-standard-4"

TRAIN_COMPUTE = MACHINE_TYPE
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = MACHINE_TYPE
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，您先看一下如何为自定义培训工作组装 Python 包。当解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将该包安装到 Docker 镜像的操作环境中的指南。

文件 `trainer/task.py` 是用于执行自定义培训工作的 Python 脚本。*注意*，当我们在工作组规范中提到它时，我们会将目录斜杠替换为点号 (`trainer.task`) 并丢弃文件后缀 (`.py`)。

#### 包组装

在下面的单元格中，您会组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您可以编写超参数调优脚本 task.py 的内容。我不会详细解释，只是供您参考。总结如下：

- 解析命令行参数，用于当前试验的超参数设置。
- 从命令行获取保存模型 artifact 的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 下载并预处理波士顿房价数据集。
- 构建一个深度神经网络模型。
- 在构建和编译模型时使用每个密集层的单元数和学习率超参数值。
- 定义一个名为 `HPTCallback` 的回调，该回调在每个 epoch 结束时获取验证损失，并通过 `hpt.report_hyperparameter_tuning_metric()` 报告给超参数调优服务。
- 使用 `fit()` 方法训练模型，并指定一个回调函数，将验证损失报告给超参数调优服务。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr, decay=args.decay))
  return model


model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

训练一个模型

### [training.using-hyperparamter-tuning](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)

### [使用超参数调整进行训练](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这告诉Vertex需要为训练提供什么类型的机器实例。
- `machine_type`：要提供的GCP实例的类型--例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用GPU；否则表示您使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

###准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉Vertex在每个训练机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您需要为自定义训练任务定义工作池规范。工作池规范包括以下内容：

- `replica_count`：要提供的此机器类型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及用于Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练任务配置的docker镜像。

- `package_uris`：这是要在已提供实例上安装的Python训练包的位置（URIs）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单个python文件或整个包的zip（存档）。在后一种情况下，作业服务将解压（解档）内容到docker镜像中。

- `python_module`：要调用以运行自定义训练任务的Python模块（脚本）。在本例中，您将调用`trainer.task.py` -- 注意，不需要添加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在这个例子中，您将设置：
 - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。有两种方法告诉训练脚本保存模型工件的位置：
     - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
     - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告知服务模型工件的位置。
 - `"--epochs=" + EPOCHS`：训练的epoch数。
 - `"--steps=" + STEPS`：每个epoch的步骤（批次）数。
 - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单台计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。 

In [ ]:
JOB_NAME = "custom_job_" + UUID
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

创建自定义作业

## 创建自定义作业

使用`CustomJob`类来创建自定义作业，比如用于超参数调优，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应虚拟机实例的规格。

In [ ]:
job = aip.CustomJob(display_name="boston_" + UUID, worker_pool_specs=worker_pool_spec)

# print(job)

## 创建一个超参数调整作业

使用`HyperparameterTuningJob`类创建一个超参数调整作业，具有以下参数：

- `display_name`：自定义作业的可阅读名称。
- `custom_job`：该自定义作业的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`：要优化的指标。字典键是指标ID，由您的训练作业报告，字典值是指标的优化目标（'最小化'或'最大化'）。
- `parameter_spec`：要优化的参数。字典键是指标ID，作为命令行关键字参数传递给您的训练作业，字典值是指标的参数规范。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="boston_" + UUID,
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
        "units": hpt.IntegerParameterSpec(min=4, max=128, scale="linear"),
    },
    max_trial_count=6,
    parallel_trial_count=1,
)

# print(hpt_job)

## 运行超参数调整任务

使用`run()`方法来执行超参数调整任务。

In [ ]:
hpt_job.run()

*示例输出:*

    INFO:google.cloud.aiplatform.jobs:创建超参数调整作业
    INFO:google.cloud.aiplatform.jobs:已创建超参数调整作业。资源名称: projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048
    INFO:google.cloud.aiplatform.jobs:要在另一个会话中使用此超参数调整作业:
    INFO:google.cloud.aiplatform.jobs:hpt_job = aiplatform.HyperparameterTuningJob.get('projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048')
    INFO:google.cloud.aiplatform.jobs:查看超参数调整作业:
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/760969798560514048?project=759209241365
    INFO:google.cloud.aiplatform.jobs:超参数调整作业 projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048 当前状态:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:超参数调整作业 projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048 当前状态:
    JobState.JOB_STATE_RUNNING
    ...
    INFO:google.cloud.aiplatform.jobs:超参数调整作业 projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048 当前状态:
    JobState.JOB_STATE_SUCCEEDED
    INFO:google.cloud.aiplatform.jobs:超参数调整作业运行完成。资源名称: projects/759209241365/locations/us-central1/hyperparameterTuningJobs/760969798560514048

显示超参数调整作业的试验结果

在超参数调整作业完成后，属性`trials`会返回每个试验的结果。

In [ ]:
print(hpt_job.trials)

*示例输出：*

    [id: "1"
    state: SUCCEEDED
    parameters {
      parameter_id: "lr"
      value {
        number_value: 0.010000000000000002
      }
    }
    parameters {
      parameter_id: "units"
      value {
        number_value: 66.0
      }
    }
    final_measurement {
      step_count: 19
      metrics {
        metric_id: "val_loss"
        value: 24.61802691948123
      }
    }
    start_time {
      seconds: 1629414344
      nanos: 232151761
    }
    end_time {
      seconds: 1629414825
    }
    , id: "2"
    state: SUCCEEDED
    parameters {
      parameter_id: "lr"
      value {
        number_value: 0.0036327633937958217
      }
    }
    parameters {
      parameter_id: "units"
      value {
        number_value: 94.0
      }
    }
    final_measurement {
      step_count: 19
      metrics {
        metric_id: "val_loss"
        value: 25.499705989186356
      }
    }
    start_time {
      seconds: 1629414960
      nanos: 618333580
    }
    end_time {
      seconds: 1629415021
    }
    ...
    , id: "6"
    state: SUCCEEDED
    parameters {
      parameter_id: "lr"
      value {
        number_value: 0.002775175422799751
      }
    }
    parameters {
      parameter_id: "units"
      value {
        number_value: 104.0
      }
    }
    final_measurement {
      step_count: 7
      metrics {
        metric_id: "val_loss"
        value: 24.655450123112377
      }
    }
    start_time {
      seconds: 1629415735
      nanos: 998711852
    }
    end_time {
      seconds: 1629415766
    }
    ]

# 清理工作

为了清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

# Delete the training job
try:
    job.delete()
except Exception as e:
    print(e)

# Delete the HPT job using the Vertex batch prediction object
hpt_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI
